### Baseado em https://github.com/nzw0301/keras-examples/blob/master/NNLM.ipynb

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import numpy as np
np.random.seed(13)

Using TensorFlow backend.


In [2]:
#path = get_file('alice.txt', origin='http://www.gutenberg.org/cache/epub/11/pg11.txt')
path = get_file('PROVERBIOS_20.txt', origin='http://www.manancialvox.com/fontedepaz/20-PROVERBIOS.txt')
doc = open(path).readlines()[3:10]

In [3]:
doc[:10]

['\x7f2 Estes provérbios nos ajudam a dar valor à sabedoria e aos bons\n',
 'conselhos e a entender os pensamentos mais profundos.\n',
 '\x7f3 Eles nos ensinam a vivermos de maneira inteligente e a sermos\n',
 'corretos, justos e honestos.\n',
 '\x7f4 Podem também tornar sábia uma pessoa sem experiência e ensinar\n',
 'os moços a serem ajuizados.\n',
 '\x7f5 Estes provérbios aumentam a sabedoria dos sábios e orientam os\n']

In [4]:
tokenizer = Tokenizer()

In [5]:
tokenizer

In [6]:
tokenizer.fit_on_texts(doc)

In [7]:
doc = tokenizer.texts_to_sequences(doc)

In [8]:
doc[:10]

[[48, 6, 4, 7, 8, 2, 42, 29, 40, 5, 1, 9, 36],
 [21, 1, 2, 16, 3, 47, 15, 43],
 [18, 46, 7, 24, 2, 32, 19, 14, 11, 1, 2, 45],
 [38, 30, 1, 28],
 [41, 12, 25, 34, 10, 31, 22, 23, 35, 1, 37],
 [3, 26, 2, 20, 13],
 [39, 6, 4, 33, 2, 5, 44, 17, 1, 27, 3]]

In [9]:
doc = [l for l in doc if len(l) > 1]

In [10]:
doc[:10]

[[48, 6, 4, 7, 8, 2, 42, 29, 40, 5, 1, 9, 36],
 [21, 1, 2, 16, 3, 47, 15, 43],
 [18, 46, 7, 24, 2, 32, 19, 14, 11, 1, 2, 45],
 [38, 30, 1, 28],
 [41, 12, 25, 34, 10, 31, 22, 23, 35, 1, 37],
 [3, 26, 2, 20, 13],
 [39, 6, 4, 33, 2, 5, 44, 17, 1, 27, 3]]

In [11]:
words_size = sum([len(words) - 1 for words in doc])

In [12]:
words_size

57

In [13]:
maxlen = max([len(x)-1 for x in doc])

In [14]:
maxlen

12

In [15]:
vocab_size = len(tokenizer.word_index) + 1

In [16]:
vocab_size

49

In [28]:
def generate_data(X, maxlen, V):
    for sentence in X: 
        #print('sentence:',sentence)
        inputs = []
        targets = []
        for i in range(1, len(sentence)):
            inputs.append(sentence[0:i])
            targets.append(sentence[i])
        y = np_utils.to_categorical(targets, V)
        inputs_sequence = sequence.pad_sequences(inputs, maxlen=maxlen)
        yield (inputs_sequence, y)

In [18]:
def sample(p):
    p /= sum(p)
    return np.where(np.random.multinomial(1, p, 1)==1)[1][0]

In [19]:
vocab_size,128,maxlen

(49, 128, 12)

In [20]:
for x_1, y_1 in generate_data(doc, maxlen, vocab_size):
    print(x_1,y_1)
    print("_________________________________________________________________________________")

sentence: [48, 6, 4, 7, 8, 2, 42, 29, 40, 5, 1, 9, 36]
[[ 0  0  0  0  0  0  0  0  0  0  0 48]
 [ 0  0  0  0  0  0  0  0  0  0 48  6]
 [ 0  0  0  0  0  0  0  0  0 48  6  4]
 [ 0  0  0  0  0  0  0  0 48  6  4  7]
 [ 0  0  0  0  0  0  0 48  6  4  7  8]
 [ 0  0  0  0  0  0 48  6  4  7  8  2]
 [ 0  0  0  0  0 48  6  4  7  8  2 42]
 [ 0  0  0  0 48  6  4  7  8  2 42 29]
 [ 0  0  0 48  6  4  7  8  2 42 29 40]
 [ 0  0 48  6  4  7  8  2 42 29 40  5]
 [ 0 48  6  4  7  8  2 42 29 40  5  1]
 [48  6  4  7  8  2 42 29 40  5  1  9]] [[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [21]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=maxlen))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [32]:
for i in range(10):
    for x, y in generate_data(doc, maxlen, vocab_size):
        model.train_on_batch(x, y)

    in_words = "pv:"
    for _ in range(maxlen):
        in_sequence = sequence.pad_sequences(tokenizer.texts_to_sequences([in_words]), maxlen=maxlen)
        wordid = sample(model.predict(in_sequence)[0])
        for k, v in tokenizer.word_index.items():
            if v == wordid:
                in_words += ' ' + k
                break

    print(i, in_words)

0 pv: podem justos aumentam a os também sábios a sem e orientam orientam
1 pv: os sábia a serem serem inteligente uma sabedoria e e e valor
2 pv: e estes a a nos sábios experiência os e ajudam orientam 2
3 pv: e justos moços honestos de ensinar sábia sábia profundos e e sabedoria
4 pv: justos aumentam eles a serem pensamentos mais eles sabedoria experiência ensinar os
5 pv: honestos tornar mais pensamentos dar sabedoria a inteligente profundos podem aos eles
6 pv: e a provérbios a de mais dos e à sermos aos 2
7 pv: estes e a ajudam dos de experiência e e inteligente sermos
8 pv: justos e eles podem pensamentos a pensamentos e ensinar e aos aos
9 pv: estes honestos sábia de dos pensamentos sábios e e sem a dar


In [23]:
in_sequence = sequence.pad_sequences(tokenizer.texts_to_sequences([in_words]), maxlen=maxlen)

In [24]:
in_sequence

array([[48, 13,  2,  9, 18,  1, 26, 10, 44, 43,  3, 28]])

In [25]:
in_words

'salmos \x7f2 ajuizados a aos \x7f3 e moços sábia dos profundos os honestos'

In [26]:
in_sequence

array([[48, 13,  2,  9, 18,  1, 26, 10, 44, 43,  3, 28]])

In [27]:
wordid

28

In [ ]:
tokenizer.word_index.items()